In [2]:
#helper libraries
import numpy as np
import pandas
import datetime as dt
import matplotlib.pyplot as plt
import scipy.fftpack
from scipy import signal
import scipy.interpolate
import h5py
from IPython.display import HTML
from geopy import distance

In [3]:
def plot_color(ax):
    ax.xaxis.label.set_color('white')
    ax.yaxis.label.set_color('white')
    ax.tick_params(axis='both', colors='white')

In [4]:
def import_csv(bagname):
    bag_df = pandas.read_csv(bagname)
    bag_df.rename(columns={'Unnamed: 0':'time'}, inplace=True) #specific to 28_07 experiments time column
    bag_df.columns=bag_df.columns.str.replace(r"/", "_") #specific to 28_07 experiments time column
    #start_time=bag_df.loc[bag_df.index[0], 'time']
    #bag_df.time=(bag_df.time-start_time)
    #bag_df=bag_df.interpolate()
    return bag_df

In [5]:
def uncooperative_hdf5(filename, key='first_key'):
    f = h5py.File(filename,'r')
    all_keys = list(f.keys())
    
    if key == 'first_key':
        print('Dataset contains these keys: ')
        print(all_keys)
        key = all_keys[0]
        print('Using only the first key: ', key)
        
    data = f[key][()]
    dic = {}
    for column_label in data.dtype.fields.keys():
        dic.setdefault(column_label, data[column_label])
    df = pandas.DataFrame(dic)
    return df

In [6]:
def epoch_setup(df):
    a=dt.datetime.utcfromtimestamp(0)
    dtx = []
    #converting timestamp to datetime format; subtracting from epoch and converting back to seconds
    for i in range (len(df.time)):
        dtx.append((dt.datetime.fromtimestamp(df.time[i])-a).total_seconds()) 
    df.insert(1,'epoch',dtx)
    sync_time = df.epoch-df.epoch[0]
    df.insert(2,'sync_time',sync_time)
    return df

#### GPS distance calculations

In [7]:
def haversine_distance(lat1, lon1, lat2, lon2):
   r = 6371
   phi1 = np.radians(lat1)
   phi2 = np.radians(lat2)
   delta_phi = np.radians(lat2 - lat1)
   delta_lambda = np.radians(lon2 - lon1)
   a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2
   res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1- a)))
   return res*1000

In [8]:
def normal_distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 * 1000 # meters

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) **2 + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) **2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d


In [9]:
def geodesic_distance(origin,destination):
    return (distance.geodesic(origin, destination).meters)